# Run with llama_cpp

# Init

In [ ]:
!pip install huggingface-hub>=0.17.1

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-creden

In [ ]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-q4.gguf --local-dir . --local-dir-use-symlinks False


Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf to /root/.cache/huggingface/hub/tmpg30acsy8
Phi-3-mini-4k-instruct-q4.gguf: 100% 2.32G/2.32G [00:31<00:00, 73.0MB/s]
./Phi-3-mini-4k-instruct-q4.gguf


In [ ]:
!apt-get update;
!wget https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64 -O cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-10-0-local-10.0.130-410.48_1.0-1_amd64.deb
!apt-key add /var/cuda-repo-10-0-local/7fa2af80.pub
!apt-get update
!apt-get -y install gcc-7 g++-7
!apt-get -y install cuda

!export PATH=/usr/local/cuda/bin${PATH:+:${PATH}}
!export LD_LIBRARY_PATH=/usr/local/cuda/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}

Get:1 file:/var/cuda-repo-10-0-local-10.0.130-410.48  InRelease
Ign:1 file:/var/cuda-repo-10-0-local-10.0.130-410.48  InRelease
Get:2 file:/var/cuda-repo-10-0-local-10.0.130-410.48  Release [574 B]
Get:2 file:/var/cuda-repo-10-0-local-10.0.130-410.48  Release [574 B]
Get:3 file:/var/cuda-repo-10-0-local-10.0.130-410.48  Release.gpg [819 B]
Get:3 file:/var/cuda-repo-10-0-local-10.0.130-410.48  Release.gpg [819 B]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:3 file:/var/cuda-repo-10-0-local-10.0.130-410.48  Release.gpg
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy

In [ ]:
! CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

In [ ]:
from llama_cpp import Llama

In [ ]:
llm = Llama(
  model_path="./Phi-3-mini-4k-instruct-q4.gguf",  # path to GGUF file
  n_ctx=4096,  # The max sequence length to use - note that longer sequence lengths require much more resources
  n_threads=8, # The number of CPU threads to use, tailor to your system and the resulting performance
  n_gpu_layers=35, # The number of layers to offload to GPU, if you have GPU acceleration available. Set to 0 if no GPU acceleration is available on your system.
  verbose = False,
)

# Cleaning GPU mem

In [ ]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()

# Test

## Basic test

In [ ]:
prompt = "How to explain Internet to a medieval knight? 500 words"

In [ ]:
# Simple inference example
output = llm(
  f"<|user|>\n{prompt}<|end|>\n<|assistant|>",
  max_tokens=4096,  # Generate up to 256 tokens
  stop=["<|end|>"],
  echo=True,  # Whether to echo the prompt
)


llama_print_timings:        load time =     204.09 ms
llama_print_timings:      sample time =     432.77 ms /   851 runs   (    0.51 ms per token,  1966.42 tokens per second)
llama_print_timings: prompt eval time =     203.81 ms /    21 tokens (    9.71 ms per token,   103.04 tokens per second)
llama_print_timings:        eval time =   13674.80 ms /   850 runs   (   16.09 ms per token,    62.16 tokens per second)
llama_print_timings:       total time =   17117.12 ms /   871 tokens


In [ ]:
output

{'id': 'cmpl-4eb65e2c-8ac7-4859-adc4-e99f610e6a60',
 'object': 'text_completion',
 'created': 1714121331,
 'model': './Phi-3-mini-4k-instruct-q4.gguf',
 'choices': [{'text': '<|user|>\nHow to explain Internet to a medieval knight? 500 words<|end|>\n<|assistant|> Hail, noble Sir, I have been entrusted with an important task - explaining the wonders of this remarkable creation called the "Internet." Though it may seem impossible to comprehend at first glance, allow me to guide you through its intricacies and marvels.\n\nThe Internet, dear knight, is a vast network of interconnected realms that exists beyond our physical world, in an ethereal plane accessible only by those with the necessary knowledge and tools. This magical web connects people from all corners of the globe - kings, scholars, peasants, and even jesters alike - enabling them to share information, communicate, and perform tasks that were once thought impossible.\n\nImagine if you will, a great library containing every book 

In [ ]:
print(output['choices'][0]['text'])

<|user|>
How to explain Internet to a medieval knight?<|end|>
<|assistant|> My noble knight, imagine the internet as an invisible web of knowledge and connection that spans across lands far and wide. It is like having access to every scroll and tome in all the kingdoms combined without ever needing to leave your chamber. Messengers would carry messages through this web at unimaginable speeds, connecting people with distant tales, trade information, and allowing scholars from different realms to share their wisdom. This magical network enables communication between individuals who are not in the same room, making it easier for kingdoms to work together towards peace and prosperity. However, like any powerful magic, one must use it wisely and respectfully, ensuring that its benefits do not come at a cost to honor or the well-being of others.


In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Test with actual data

In [ ]:
snt = "Despite the enthusiastic rhetoric about the so called collective intelligence, conspiracy theories – e.g. global warming induced by chemtrails or the link between vaccines and autism – find on the Web a natural medium for their dissemination. Users preferentially consume information according to their system of beliefs and the strife within users of opposite worldviews (e.g., scientific and conspiracist) may result in heated debates. In this work we provide a genuine example of information consumption on a set of 1.2 million of Facebook Italian users. We show by means of a thorough quantitative analysis that information supporting different worldviews – i.e. scientific and conspiracist news – are consumed in a comparable way. Moreover, we measure the effect of 4709 evidently false information (satirical version of conspiracist stories) and 4502 debunking memes (information aiming at contrasting unsubstantiated rumors) on polarized users of conspiracy claims."

prompt = "Simplify the following sentence: '" + snt + "' In doing that, focus on both the lexical aspect of simplification (using more common words) and the syntactical aspect. Sentences should be very short and to the point. Wherever there is a comma split the sentence into two sentences, there should be no comma in the final text. Say only one thing per sentence."
print(prompt)

Simplify the following sentence: 'Despite the enthusiastic rhetoric about the so called collective intelligence, conspiracy theories – e.g. global warming induced by chemtrails or the link between vaccines and autism – find on the Web a natural medium for their dissemination. Users preferentially consume information according to their system of beliefs and the strife within users of opposite worldviews (e.g., scientific and conspiracist) may result in heated debates. In this work we provide a genuine example of information consumption on a set of 1.2 million of Facebook Italian users. We show by means of a thorough quantitative analysis that information supporting different worldviews – i.e. scientific and conspiracist news – are consumed in a comparable way. Moreover, we measure the effect of 4709 evidently false information (satirical version of conspiracist stories) and 4502 debunking memes (information aiming at contrasting unsubstantiated rumors) on polarized users of conspiracy c

In [ ]:
output = llm(
  prompt,
  max_tokens=2048,
  temperature=0,
  stop=["<|end|>"]
)
print(output['choices'][0]['text'])


llama_print_timings:        load time =     272.49 ms
llama_print_timings:      sample time =       1.16 ms /     2 runs   (    0.58 ms per token,  1719.69 tokens per second)
llama_print_timings: prompt eval time =     271.99 ms /   307 tokens (    0.89 ms per token,  1128.73 tokens per second)
llama_print_timings:        eval time =      18.28 ms /     1 runs   (   18.28 ms per token,    54.71 tokens per second)
llama_print_timings:       total time =     297.57 ms /   308 tokens


In [ ]:
output

{'id': 'cmpl-f765352f-400a-403e-858a-16cc8fe01cd6',
 'object': 'text_completion',
 'created': 1714047774,
 'model': './Phi-3-mini-4k-instruct-q4.gguf',
 'choices': [{'text': "\n<|assistant|> Collective intelligence talk overshadows how conspiracy theories spread on the web, with users often choosing information that aligns with their beliefs. This leads to conflicts between scientific and conspiratorial worldviews. An analysis of 1.2 million Italian Facebook users showed similar consumption rates for supportive news from both perspectives. Additionally, false and debunking content had varying impacts on believers in these theories.\n\nSplit the sentence into two:\n\n- Conspiracy theories spread online as web platforms offer them natural dissemination channels.\n\n- These beliefs trigger conflicts between scientific and conspiratorial views among users.\n\n- A study of 1.2 million Italian Facebook users found no significant difference in consuming news from either viewpoint.\n\n- False 


```
"Despite the enthusiastic rhetoric about the so called collective intelligence, conspiracy theories – e.g. global warming induced by chemtrails or the link between vaccines and autism – find on the Web a natural medium for their dissemination. Users preferentially consume information according to their system of beliefs and the strife within users of opposite worldviews (e.g., scientific and conspiracist) may result in heated debates. In this work we provide a genuine example of information consumption on a set of 1.2 million of Facebook Italian users. We show by means of a thorough quantitative analysis that information supporting different worldviews – i.e. scientific and conspiracist news – are consumed in a comparable way. Moreover, we measure the effect of 4709 evidently false information (satirical version of conspiracist stories) and 4502 debunking memes (information aiming at contrasting unsubstantiated rumors) on polarized users of conspiracy claims.
```


In [ ]:
prompt2 = "Take the following text and split the sentences as much as you can, each sentence must say only one thing: '" + output['choices'][0]['text'] + "' "
print(prompt2)

Take the following text and split the sentences as much as you can, each sentence must say only one thing: '
<|assistant|> Collective intelligence enthusiasm doesn't prevent conspiracy theories from spreading online, such as global warming chemtrails or vaccines causing autism. People often choose information that aligns with their beliefs, leading to intense debates between scientific and conspiracist viewpoints. This study analyzes Facebook users in Italy, showing equal consumption of both types of news. It also examines the impact of false conspiracy stories and debunking memes on those who believe them.' 


In [ ]:
output = llm(
  prompt2,
  max_tokens=2048,
  temperature=0,
  stop=["<|end|>"]
)
print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =     200.45 ms /   362 runs   (    0.55 ms per token,  1805.90 tokens per second)
llama_print_timings: prompt eval time =     198.66 ms /   131 tokens (    1.52 ms per token,   659.42 tokens per second)
llama_print_timings:        eval time =    5524.43 ms /   361 runs   (   15.30 ms per token,    65.35 tokens per second)
llama_print_timings:       total time =    6922.61 ms /   492 tokens



<|assistant|> Collective intelligence enthusiasm doesn't prevent conspiracy theories from spreading online. Global warming chemtrails or vaccines causing autism are examples of such theories. People often choose information that aligns with their beliefs, leading to intense debates between scientific and conspiricist viewpoints. The study analyzes Facebook users in Italy. It shows equal consumption of both types of news among these users. The impact of false conspiracy stories on believers is also examined. Debunking memes' effectiveness is analyzed as well.
<|assistant|> This study focuses on the spread of conspiracy theories online, despite collective intelligence enthusiasm. Examples include global warming chemtrails and vaccines causing autism. Belief-aligned information consumption leads to debates between scientific facts and conspiracy beliefs. The research involves Facebook users in Italy, revealing their equal intake of different news types. It also investigates the influence

In [ ]:
prompt3 = "Take the following text and split the sentences as much as you can, each sentence must say only one thing: '" + snt + "' "
print(prompt3)

Take the following text and split the sentences as much as you can, each sentence must say only one thing: 'Despite the enthusiastic rhetoric about the so called collective intelligence, conspiracy theories – e.g. global warming induced by chemtrails or the link between vaccines and autism – find on the Web a natural medium for their dissemination. Users preferentially consume information according to their system of beliefs and the strife within users of opposite worldviews (e.g., scientific and conspiracist) may result in heated debates. In this work we provide a genuine example of information consumption on a set of 1.2 million of Facebook Italian users. We show by means of a thorough quantitative analysis that information supporting different worldviews – i.e. scientific and conspiracist news – are consumed in a comparable way. Moreover, we measure the effect of 4709 evidently false information (satirical version of conspiracist stories) and 4502 debunking memes (information aiming

In [ ]:
output = llm(
  prompt3,
  max_tokens=2048,
  temperature=0
)
print(output['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =     107.13 ms /   190 runs   (    0.56 ms per token,  1773.50 tokens per second)
llama_print_timings: prompt eval time =     254.28 ms /   232 tokens (    1.10 ms per token,   912.39 tokens per second)
llama_print_timings:        eval time =    3074.06 ms /   189 runs   (   16.26 ms per token,    61.48 tokens per second)
llama_print_timings:       total time =    3953.21 ms /   421 tokens


<|assistant|> 1. Despite enthusiastic rhetranter about collective intelligence, conspiracy theories find a natural medium for dissemination on the web.
2. Users consume information according to their beliefs system.
3. Strife between scientific and conspiracist users may result in heated debates.
4. This work provides an example of information consumption among 1.2 million Facebook Italian users.
5. Information supporting different worldviews is consumed comparably.
6. The study measures the effect of false information on polarized conspiracy claimers.
7. There are 4,709 satirical versions of conspiracist stories.
8. Debunking memes number 4,502 and aim to contrast unsubstantiated rumors.
9. The study measures the effect of debunking memes on polarized users.


In [ ]:
prompt4 = "Take the following text'" + snt + "' list all the logic propositions contained in that text seperately and seperate them with <c>"
print(prompt4)

Take the following text'Despite the enthusiastic rhetoric about the so called collective intelligence, conspiracy theories – e.g. global warming induced by chemtrails or the link between vaccines and autism – find on the Web a natural medium for their dissemination. Users preferentially consume information according to their system of beliefs and the strife within users of opposite worldviews (e.g., scientific and conspiracist) may result in heated debates. In this work we provide a genuine example of information consumption on a set of 1.2 million of Facebook Italian users. We show by means of a thorough quantitative analysis that information supporting different worldviews – i.e. scientific and conspiracist news – are consumed in a comparable way. Moreover, we measure the effect of 4709 evidently false information (satirical version of conspiracist stories) and 4502 debunking memes (information aiming at contrasting unsubstantiated rumors) on polarized users of conspiracy claims.' li

In [ ]:
output4 = llm(
  prompt4,
  max_tokens=2048,
  temperature=0,
  stop=["<|end|>"]
)
print(output4['choices'][0]['text'])

Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =     261.79 ms /   467 runs   (    0.56 ms per token,  1783.87 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    7768.49 ms /   467 runs   (   16.63 ms per token,    60.11 tokens per second)
llama_print_timings:       total time =    9524.76 ms /   468 tokens



<|assistant|> <c>Despite the enthusiastic rhetranter about the so called collective intelligence, conspiracy theories exist – e.g., global warming induced by chemtrails or the link between vaccines and autism – find on the Web a natural medium for their dissemination.</c><c>Users preferentially consume information according to their system of beliefs.</c><c>The strife within users of opposite worldviews (e.g., scientific and conspiracist) may result in heated debates.</c><c>In this work, we provide a genuine example of information consumption on a set of 1.2 million Facebook Italian users.</c><c>We show by means of a thorough quantitative analysis that information supporting different worldviews – i.e., scientific and conspiracist news – are consumed in a comparable way.</c><c>Moreover, we measure the effect of 4709 evidently false information (satirical version of conspiracist stories) and 4502 debunking memes on polarized users of conspiracy claims.</c>

Note: The text does not expl

In [ ]:
prompt41 = "Take the following text'" + output4['choices'][0]['text'] + "' remove all unnecessary word from each sentence."
print(prompt41)
output41 = llm(
  prompt41,
  max_tokens=2048,
  temperature=0,
  stop=["<|end|>"]
)
print(output41['choices'][0]['text'])

Llama.generate: prefix-match hit


Take the following text'.
<|assistant|> 1. Enthusiastic rhetrancy about collective intelligence exists.
2. Conspiracy theories exist, such as global warming induced by chemtrails or the link between vaccines and autism.
3. The Web provides a natural medium for the dissemination of conspiracy theories.
4. Users consume information according to their system of beliefs.
5. Strife within users with opposite worldviews may result in heated debates.
6. An example of information consumption on 1.2 million Facebook Italian users is provided.
7. Information supporting different worldviews (scientific and conspiracist news) are consumed comparably.
8. There are 4709 evidently false information (satirical version of conspiracist stories).
9. There are 4502 debunking memes aiming at contrasting unsubstantiated rumors.
10. Polarized users of conspiracy claims exist.

Note: The text does not contain explicit logical propositions in the form of "if-then" statements or other formal logic structures, b


llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =      80.57 ms /   147 runs   (    0.55 ms per token,  1824.41 tokens per second)
llama_print_timings: prompt eval time =     214.82 ms /   275 tokens (    0.78 ms per token,  1280.16 tokens per second)
llama_print_timings:        eval time =    2299.63 ms /   146 runs   (   15.75 ms per token,    63.49 tokens per second)
llama_print_timings:       total time =    3015.71 ms /   421 tokens



<|assistant|> 1. Collective intelligence enthusiasm exists.
2. Conspiracy theories exist (chemtrails, vaccines-autism).
3. Web disseminates conspiracy theories.
4. Users consume info by beliefs.
5. Opposing worldviews lead to debates.
6. Information consumption study on 1.2M Facebook Italians.
7. Comparable intake of scientific and conspiracist news.
8. 4709 false, satirical conspiracy stories.
9. 4502 debunking memes.
10. Conspiracy claim polarization exists.


In [ ]:
prompt42 = "Take the following text'" + output41['choices'][0]['text'] + "'Take each item in the list and append them without modifying them:"
print(prompt42)
output42 = llm(
  prompt42,
  max_tokens=2048,
  temperature=0,
  stop=["<|end|>"]
)
print(output42['choices'][0]['text'])

Take the following text'
<|assistant|> 1. Collective intelligence enthusiasm exists.
2. Conspiracy theories exist (chemtrails, vaccines-autism).
3. Web disseminates conspiracy theories.
4. Users consume info by beliefs.
5. Opposing worldviews lead to debates.
6. Information consumption study on 1.2M Facebook Italians.
7. Comparable intake of scientific and conspiracist news.
8. 4709 false, satirical conspiracy stories.
9. 4502 debunking memes.
10. Conspiracy claim polarization exists.'Take each item in the list and append them without modifying them:


Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =      91.11 ms /   159 runs   (    0.57 ms per token,  1745.18 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    2494.36 ms /   159 runs   (   15.69 ms per token,    63.74 tokens per second)
llama_print_timings:       total time =    3056.70 ms /   160 tokens


 The collected data shows that there is a significant interest in collective intelligence among individuals. However, this interest coexists with the presence of various conspiracy theories related to chemtrails and vaccines causing autism. These theories are often spread through online platforms such as Facebook, where users tend to consume information based on their pre-existing beliefs. This consumption leads to heated debates between those who hold opposing worldviews. A study conducted on 1.2 million Italian Facebook users revealed that there is a comparable intake of scientific and conspiracist news. The data also indicates the presence of approximately 4709 false or satirical conspiracy stories, while around 4502 debunking memes were identified.'


In [ ]:
def simp_strat(snt):

  outputs = []

  prompt = "Take the following text'" + snt + "' list all the logic propositions contained in that text seperately"
  output = llm(
    prompt,
    max_tokens=2048,
    temperature=0,
    stop=["<|end|>"]
  )

  outputs.append(output['choices'][0]['text'])

  prompt = "Take the following text'" + output['choices'][0]['text'] + "' remove all unnecessary word from each sentence."

  output = llm(
    prompt,
    max_tokens=2048,
    temperature=0,
    stop=["<|end|>"]
  )

  outputs.append(output['choices'][0]['text'])

  prompt = "<|query|>Take the following text'" + output['choices'][0]['text'] + "'For each complicated or rare word, replace it with a simpler synonym if it exists.<|answer|>"

  output = llm(
    prompt,
    max_tokens=2048,
    temperature=0,
    stop=["<|end|>"]
  )

  outputs.append(output['choices'][0]['text'])


  prompt = "<|query|>Take each item in the following text and append them without modifying them:'" + output['choices'][0]['text'] + "'<|answer|>"

  output = llm(
    prompt,
    max_tokens=2048,
    temperature=0,
    stop=["<|end|>"]
  )

  outputs.append(output['choices'][0]['text'])

  return outputs

In [ ]:
output = simp_strat(snt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =     158.25 ms /   268 runs   (    0.59 ms per token,  1693.50 tokens per second)
llama_print_timings: prompt eval time =     258.36 ms /   247 tokens (    1.05 ms per token,   956.02 tokens per second)
llama_print_timings:        eval time =    4349.52 ms /   267 runs   (   16.29 ms per token,    61.39 tokens per second)
llama_print_timings:       total time =    5650.36 ms /   514 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =      85.66 ms /   155 runs   (    0.55 ms per token,  1809.50 tokens per second)
llama_print_timings: prompt eval time =     236.28 ms /   275 tokens (    0.86 ms per token,  1163.90 tokens per second)
llama_print_timings:        eval time =    2548.01 ms /   154 runs   (   16.55 ms per token,    60.44 tokens per second)
llama_print_timings:       to

In [ ]:
output[3]

" The text highlights several key points regarding the role of social media in spreading misinformation:\n\n1. Group interest in shared knowledge is strong, which may contribute to users' susceptibility to conspiracy theories as they seek validation from their peers.\n2. Conspiracy theories are present on social media platforms like Facebook and can be fueled by the internet's ability to disseminate such content rapidly.\n3. People tend to choose information that aligns with their existing beliefs, which may lead them to accept misinformation more readily if it confirms their preconceived notions.\n4. The example of 1.2 million Facebook users in Italy demonstrates the scale at which social media can influence public opinion and potentially contribute to polarization based on shared information or conspiracy theories.\n5. People's acceptance of different types of information, such as scientific versus conspiratorial content, highlights how individuals may be more receptive to misinforma

In [ ]:
import re
def remove_double_spaces_and_newlines(text):
    # Remove double spaces
    text = re.sub(r'\s+', ' ', text)
    # Remove newline characters
    text = text.replace('\n', '')
    return text

In [ ]:
def simp_strat_few_shot(snt):

  outputs = []

  prompt = """Take a text list all the smallest logic propositions contained in that text seperately while keeping all of the relevent information.
              <|query|>
                Information provided by whistleblower Edward Snowden imposingly demonstrated the
                advanced capabilities of intelligence agencies, especially the National
                Security Agency (NSA), to monitor Internet usage on a large scale.
              <|answer|>Edward Snowden is a whistleblower.
                He provided information.
                They demonstrated the capabilities of intelligence agencies.
                The National Security Agency (NSA) is one of them.
                They can monitor internet usage.
                They can do it on a large scale.
              <|end|>
              <|query|>"""+ snt + "<|answer|>"
  prompt = remove_double_spaces_and_newlines(prompt)
  output = llm(
    prompt,
    max_tokens=2048,
    temperature=0,
    stop=["<|end|>"]
  )

  outputs.append(output['choices'][0]['text'])


  prompt = """Take a text remove complicated word and replace them with a simpler synonym.
              <|query|>
                Rabbits often feed on young, tender perennial growth as it emerges in spring.
                Performance test for a system coupled with a locally manufactured station engine model MWM will start shortly.
                Perhaps the effect of West Nile Virus is sufficient to extinguish endemic birds already severely stressed by habitat losses.
              <|answer|>
                Rabbits often eat young and soft plants as it grows in spring, or on young transplants.
                Performance test for a system mixed with a locally made station engine model MWM will start soon.
                Maybe the effect of West Nile Virus is enough to get rid of endemic birds already very stressed by loss of habitat.
              <|end|>
              <|query|>"""+ output['choices'][0]['text'] + "<|answer|>"
  prompt = remove_double_spaces_and_newlines(prompt)
  output = llm(
    prompt,
    max_tokens=2048,
    temperature=0,
    stop=["<|end|>"]
  )

  outputs.append(output['choices'][0]['text'])

  """"
  prompt = "<|query|>Take the following text'" + output['choices'][0]['text'] + "'For each complicated or rare word, replace it with a simpler synonym if it exists.<|answer|>"

  output = llm(
    prompt,
    max_tokens=2048,
    temperature=0,
    stop=["<|end|>"]
  )

  outputs.append(output['choices'][0]['text'])


  prompt = "<|query|>Take each item in the following text and append them without modifying them:'" + output['choices'][0]['text'] + "'<|answer|>"

  output = llm(
    prompt,
    max_tokens=2048,
    temperature=0,
    stop=["<|end|>"]
  )

  outputs.append(output['choices'][0]['text'])"""

  return outputs

In [ ]:
output = simp_strat_few_shot(snt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =      74.85 ms /   113 runs   (    0.66 ms per token,  1509.63 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    1934.02 ms /   113 runs   (   17.12 ms per token,    58.43 tokens per second)
llama_print_timings:       total time =    2365.92 ms /   114 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =      84.76 ms /   133 runs   (    0.64 ms per token,  1569.12 tokens per second)
llama_print_timings: prompt eval time =     241.47 ms /   282 tokens (    0.86 ms per token,  1167.85 tokens per second)
llama_print_timings:        eval time =    2170.97 ms /   132 runs   (   16.45 ms per token,    60.80 tokens per second)
llama_print_timings:       to

In [ ]:
output

["Conspiracy theories exist. They are disseminated online. Examples include global warming induced by chemtrails or vaccines and autism link. Users consume information based on their beliefs. This can lead to heated debates between scientific and conspiracist worldviews. The study analyzed Facebook Italian users' information consumption. Both types of news (scientific and conspiracist) are consumed similarly by polarized users. False information and debunking memes were measured for their effect on these users.",
 'Some people believe in conspiracy theories, which they often find online. Examples include the idea that chemtrails cause global warming or a link between vaccines and autism. People tend to look for information that aligns with what they already think. This can result in intense arguments between those who trust science and those who believe in conspiracies. The research looked at how Italian Facebook users consume news, whether scientific or based on conspiracy theories. I

In [ ]:
snt2 = "HTTP cookies have been widely used for maintaining session states, personalizing, authenticating, and tracking user behaviors. Despite their importance and usefulness, cookies have raised public concerns on Internet privacy because they can be exploited by third-parties to track user behaviors and build user profiles. In addition, stolen cookies may also incur severe security problems. However, current Web browsers lack secure and convenient mechanisms for cookie management. A cookie management scheme, which is easy-to-use and has minimal privacy risk, is in great demand; but designing such a scheme is a challenge. In this paper, we conduct a large scale HTTP cookie measurement and introduce CookiePicker, a system that can automatically validate the usefulness of cookies from a Web site and set the cookie usage permission on behalf of users. CookiePicker helps users achieve the maximum benefit brought by cookies, while minimizing the possible privacy and security risks. We implement CookiePicker as an extension to Firefox Web browser, and obtain promising results in the experiments."

In [ ]:
output = simp_strat_few_shot(snt2)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =      59.33 ms /    92 runs   (    0.64 ms per token,  1550.62 tokens per second)
llama_print_timings: prompt eval time =     296.39 ms /   352 tokens (    0.84 ms per token,  1187.64 tokens per second)
llama_print_timings:        eval time =    1505.47 ms /    91 runs   (   16.54 ms per token,    60.45 tokens per second)
llama_print_timings:       total time =    2139.58 ms /   443 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =      63.57 ms /    97 runs   (    0.66 ms per token,  1525.97 tokens per second)
llama_print_timings: prompt eval time =     245.53 ms /   261 tokens (    0.94 ms per token,  1063.02 tokens per second)
llama_print_timings:        eval time =    1573.61 ms /    96 runs   (   16.39 ms per token,    61.01 tokens per second)
llama_print_timings:       to

In [ ]:
output

["HTTP cookies are used for session states, personalization, authentication, and tracking user behaviors. They can be exploited by third parties. Stolen cookies may cause severe security problems. Current web browsers lack secure cookie management mechanisms. CookiePicker is a system that validates the usefulness of cookies and sets usage permissions on behalf of users to minimize privacy and security risks. It's implemented as an extension for Firefox browser.",
 "Cookies are used for remembering user sessions, customizing content, logging in, and monitoring how people use websites. They can be misused by others. If cookies get stolen, it could lead to big security issues. Web browsers don't handle cookies very well yet. CookiePicker is a tool that checks if cookies are needed and lets users decide who can see them, helping protect privacy and safety. It works as an add-on for Firefox browser."]

In [ ]:
def simp_strat_few_shot_reversed_order(snt):

  outputs = []


  prompt = """Take a text remove complicated word and replace them with a simpler synonym.
              <|query|>
                Rabbits often feed on young, tender perennial growth as it emerges in spring.
                Performance test for a system coupled with a locally manufactured station engine model MWM will start shortly.
                Perhaps the effect of West Nile Virus is sufficient to extinguish endemic birds already severely stressed by habitat losses.
              <|answer|>
                Rabbits often eat young and soft plants as it grows in spring, or on young transplants.
                Performance test for a system mixed with a locally made station engine model MWM will start soon.
                Maybe the effect of West Nile Virus is enough to get rid of endemic birds already very stressed by loss of habitat.
              <|end|>
              <|query|>"""+ snt + "<|answer|>"
  prompt = remove_double_spaces_and_newlines(prompt)
  output = llm(
    prompt,
    max_tokens=2048,
    temperature=0,
    stop=["<|end|>"]
  )

  outputs.append(output['choices'][0]['text'])

  prompt = """Take a text list all the smallest logic propositions contained in that text seperately while keeping all of the relevent information.
              <|query|>
                Information provided by whistleblower Edward Snowden imposingly demonstrated the
                advanced capabilities of intelligence agencies, especially the National
                Security Agency (NSA), to monitor Internet usage on a large scale.
              <|answer|>Edward Snowden is a whistleblower.
                He provided information.
                They demonstrated the capabilities of intelligence agencies.
                The National Security Agency (NSA) is one of them.
                They can monitor internet usage.
                They can do it on a large scale.
              <|end|>
              <|query|>"""+ snt + "<|answer|>"
  prompt = remove_double_spaces_and_newlines(prompt)
  output = llm(
    prompt,
    max_tokens=2048,
    temperature=0,
    stop=["<|end|>"]
  )

  outputs.append(output['choices'][0]['text'])

  return outputs

In [ ]:
output = simp_strat_few_shot_reversed_order(snt)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =     107.82 ms /   181 runs   (    0.60 ms per token,  1678.68 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    2989.56 ms /   181 runs   (   16.52 ms per token,    60.54 tokens per second)
llama_print_timings:       total time =    3654.19 ms /   182 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =      76.43 ms /   120 runs   (    0.64 ms per token,  1570.06 tokens per second)
llama_print_timings: prompt eval time =     287.25 ms /   365 tokens (    0.79 ms per token,  1270.67 tokens per second)
llama_print_timings:        eval time =    1985.69 ms /   119 runs   (   16.69 ms per token,    59.93 tokens per second)
llama_print_timings:       to

In [ ]:
output

[" Even though there's a lot of talk about the idea that groups can act smart together, people still believe in things like global warming being caused by chemtrails or vaccines causing autism. These ideas spread easily online because they fit into what people already think. People tend to read information that agrees with their beliefs and this often leads to arguments between those who believe different things. In our study, we looked at how 1.2 million Facebook users in Italy shared news from both scientific and conspiracy perspectives. We found out through detailed analysis that people share these types of news about equally. Also, we checked the impact of over 4700 false stories (made to look like they're from conspiracies) and more than 4500 memes meant to debunk them on those who believe in conspiracy theories.",
 "Conspiracy theories exist. They are disseminated online. Examples include global warming induced by chemtrails or vaccines and autism link. Users consume information 

In [ ]:
def simp_parametrized_few_shot(snt, strat, temp = 0):

  output_list = [snt]

  def syntax_simp(snt):
    prompt = """Take a text list all the smallest logic propositions contained in that text seperately while keeping all of the relevent information.
                <|query|>
                  Information provided by whistleblower Edward Snowden imposingly demonstrated the
                  advanced capabilities of intelligence agencies, especially the National
                  Security Agency (NSA), to monitor Internet usage on a large scale.
                <|answer|>Edward Snowden is a whistleblower.
                  He provided information.
                  They demonstrated the capabilities of intelligence agencies.
                  The National Security Agency (NSA) is one of them.
                  They can monitor internet usage.
                  They can do it on a large scale.
                <|end|>
                <|query|>"""+ snt + "<|answer|>"
    prompt = remove_double_spaces_and_newlines(prompt)
    output = llm(
      prompt,
      max_tokens=2048,
      temperature=temp,
      stop=["<|end|>"]
    )
    return output['choices'][0]['text']

  def lexical_simp(snt):
    prompt = """Take a text remove complicated word and replace them with a simpler synonym.
                <|query|>
                  Rabbits often feed on young, tender perennial growth as it emerges in spring.
                  Performance test for a system coupled with a locally manufactured station engine model MWM will start shortly.
                  Perhaps the effect of West Nile Virus is sufficient to extinguish endemic birds already severely stressed by habitat losses.
                <|answer|>
                  Rabbits often eat young and soft plants as it grows in spring, or on young transplants.
                  Performance test for a system mixed with a locally made station engine model MWM will start soon.
                  Maybe the effect of West Nile Virus is enough to get rid of endemic birds already very stressed by loss of habitat.
                <|end|>
                <|query|>"""+ snt + "<|answer|>"
    prompt = remove_double_spaces_and_newlines(prompt)
    output = llm(
      prompt,
      max_tokens=2048,
      temperature=temp,
      stop=["<|end|>"]
    )
    return output['choices'][0]['text']

  for c in strat:
    if c =="s":
      output_list.append(syntax_simp(output_list[-1]))
    if c =="l":
      output_list.append(lexical_simp(output_list[-1]))

  return output_list

In [ ]:
output = simp_parametrized_few_shot(snt, "slsl")

Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =      73.07 ms /   120 runs   (    0.61 ms per token,  1642.31 tokens per second)
llama_print_timings: prompt eval time =     312.16 ms /   365 tokens (    0.86 ms per token,  1169.26 tokens per second)
llama_print_timings:        eval time =    1970.37 ms /   119 runs   (   16.56 ms per token,    60.39 tokens per second)
llama_print_timings:       total time =    2683.94 ms /   484 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =      74.92 ms /   125 runs   (    0.60 ms per token,  1668.47 tokens per second)
llama_print_timings: prompt eval time =     235.12 ms /   289 tokens (    0.81 ms per token,  1229.18 tokens per second)
llama_print_timings:        eval time =    2053.84 ms /   124 runs   (   16.56 ms per token,    60.37 tokens per second)
llama_print_timings:       to

In [ ]:
for i in output:
  print(i)
  print("\n_________________________________________________________________________________\n")

Despite the enthusiastic rhetoric about the so called collective intelligence, conspiracy theories – e.g. global warming induced by chemtrails or the link between vaccines and autism – find on the Web a natural medium for their dissemination. Users preferentially consume information according to their system of beliefs and the strife within users of opposite worldviews (e.g., scientific and conspiracist) may result in heated debates. In this work we provide a genuine example of information consumption on a set of 1.2 million of Facebook Italian users. We show by means of a thorough quantitative analysis that information supporting different worldviews – i.e. scientific and conspiracist news – are consumed in a comparable way. Moreover, we measure the effect of 4709 evidently false information (satirical version of conspiracist stories) and 4502 debunking memes (information aiming at contrasting unsubstantiated rumors) on polarized users of conspiracy claims.

__________________________

In [ ]:
output = simp_parametrized_few_shot(snt, "lsls")

Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =     103.50 ms /   181 runs   (    0.57 ms per token,  1748.74 tokens per second)
llama_print_timings: prompt eval time =     267.98 ms /   233 tokens (    1.15 ms per token,   869.47 tokens per second)
llama_print_timings:        eval time =    3014.36 ms /   180 runs   (   16.75 ms per token,    59.71 tokens per second)
llama_print_timings:       total time =    3896.80 ms /   413 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     315.14 ms
llama_print_timings:      sample time =      63.42 ms /   110 runs   (    0.58 ms per token,  1734.58 tokens per second)
llama_print_timings: prompt eval time =     264.72 ms /   316 tokens (    0.84 ms per token,  1193.70 tokens per second)
llama_print_timings:        eval time =    1842.53 ms /   109 runs   (   16.90 ms per token,    59.16 tokens per second)
llama_print_timings:       to

In [ ]:
for i in output:
  print(i)
  print("\n_________________________________________________________________________________\n")

Despite the enthusiastic rhetoric about the so called collective intelligence, conspiracy theories – e.g. global warming induced by chemtrails or the link between vaccines and autism – find on the Web a natural medium for their dissemination. Users preferentially consume information according to their system of beliefs and the strife within users of opposite worldviews (e.g., scientific and conspiracist) may result in heated debates. In this work we provide a genuine example of information consumption on a set of 1.2 million of Facebook Italian users. We show by means of a thorough quantitative analysis that information supporting different worldviews – i.e. scientific and conspiracist news – are consumed in a comparable way. Moreover, we measure the effect of 4709 evidently false information (satirical version of conspiracist stories) and 4502 debunking memes (information aiming at contrasting unsubstantiated rumors) on polarized users of conspiracy claims.

__________________________

# Benchmarking

In [ ]:
import re
def remove_double_spaces_and_newlines(text):
    # Remove double spaces
    text = re.sub(r'\s+', ' ', text)
    # Remove newline characters
    text = text.replace('\n', '')
    return text

In [ ]:
def simp_parametrized_few_shot(snt, strat, temp = 0):

  output_list = [snt]

  def syntax_simp(snt):
    prompt = """Take a text list all the smallest logic propositions contained in that text seperately while keeping all of the relevent information.
                <|query|>
                  Information provided by whistleblower Edward Snowden imposingly demonstrated the
                  advanced capabilities of intelligence agencies, especially the National
                  Security Agency (NSA), to monitor Internet usage on a large scale.
                <|answer|>Edward Snowden is a whistleblower.
                  He provided information.
                  They demonstrated the capabilities of intelligence agencies.
                  The National Security Agency (NSA) is one of them.
                  They can monitor internet usage.
                  They can do it on a large scale.
                <|end|>
                <|query|>"""+ snt + "<|answer|>"
    prompt = remove_double_spaces_and_newlines(prompt)
    output = llm(
      prompt,
      max_tokens=2048,
      temperature=temp,
      stop=["<|end|>"]
    )
    return output['choices'][0]['text']

  def lexical_simp(snt):
    prompt = """Take a text remove complicated word and replace them with a simpler synonym.
                <|query|>
                  Rabbits often feed on young, tender perennial growth as it emerges in spring.
                  Performance test for a system coupled with a locally manufactured station engine model MWM will start shortly.
                  Perhaps the effect of West Nile Virus is sufficient to extinguish endemic birds already severely stressed by habitat losses.
                <|answer|>
                  Rabbits often eat young and soft plants as it grows in spring, or on young transplants.
                  Performance test for a system mixed with a locally made station engine model MWM will start soon.
                  Maybe the effect of West Nile Virus is enough to get rid of endemic birds already very stressed by loss of habitat.
                <|end|>
                <|query|>"""+ snt + "<|answer|>"
    prompt = remove_double_spaces_and_newlines(prompt)
    output = llm(
      prompt,
      max_tokens=2048,
      temperature=temp,
      stop=["<|end|>"]
    )
    return output['choices'][0]['text']

  for c in strat:
    if c =="s":
      output_list.append(syntax_simp(output_list[-1]))
    if c =="l":
      output_list.append(lexical_simp(output_list[-1]))

  return output_list

In [ ]:
import pandas as pd

In [ ]:
reference = df = pd.read_json('/content/simpletext_task3_2024_train_snt_reference.json')
source = df = pd.read_json('/content/simpletext_task3_2024_train_snt_source.json')
data = pd.merge(reference, source, on='snt_id', how='inner')
data

,snt_id,simplified_snt,query_id,query_text,doc_id,source_snt
0,G11.1_2892036907_1,Current academic and industrial research is in...,G11.1,drones,2892036907,"In the modern era of automation and robotics, ..."
1,G11.1_2892036907_2,Drones are increasingly used in the civilian a...,G11.1,drones,2892036907,With the ever increasing number of unmanned ae...
2,G11.1_2892036907_3,Governments set guidelines on the operation ce...,G11.1,drones,2892036907,Due to guidelines set by the governments regar...
3,G11.1_2892036907_4,Researchers propose data-driven solutions allo...,G11.1,drones,2892036907,In an attempt to achieve the above mentioned t...
4,G11.1_2892036907_5,"The algorithm, based on the Inception model, d...",G11.1,drones,2892036907,Derived from the classic image classification ...
...,...,...,...,...,...,...
1083,G01.1_1549686097_2,"Users can access city maps, route planning, na...",G01.1,Digital assistant,1549686097,These services are to be accessed by users wit...
1084,G01.1_1549686097_3,"Compared to laptops or personal computers, mob...",G01.1,Digital assistant,1549686097,"Mobile end devices, however, do not have compu..."
1085,G01.1_1549686097_4,This shortage can be overcome by using specifi...,G01.1,Digital assistant,1549686097,These deficits can be bypassed by employing sp...
1086,G01.1_1549686097_5,This paper shows the gained experience from de...,G01.1,Digital assistant,1549686097,This paper outlines the experience gained in t...


In [ ]:
def generate_simp(row, strat):
  return simp_parametrized_few_shot(row["source_snt"], strat = strat )

pour avoir la progression

In [ ]:
pip install tqdm

In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="Processing")

In [ ]:
data["generated_snt_sl"] = data.progress_apply(lambda row: generate_simp(row, strat = "sl"), axis=1)

Processing: 100%|██████████| 1088/1088 [3:24:38<00:00, 11.29s/it]


In [ ]:
data["generated_snt_sl"][0]

['In the modern era of automation and robotics, autonomous vehicles are currently the focus of academic and industrial research.',
 'Autonomous vehicles are the subject of current research. The research is in both academia and industry. It relates to automation and robotics.',
 ' Self-driving cars are being studied now. This work happens at universities and companies. It has to do with making machines drive themselves and robots.']

In [ ]:
data["generated_snt_s"] = data.apply(lambda row: row["generated_snt_sl"][1], axis=1)
data["generated_snt_sl2"] = data.apply(lambda row: row["generated_snt_sl"][2], axis=1)

In [ ]:
#data.drop("generated_snt_sl", axis=1, inplace=True)
data.rename(columns={"generated_snt_sl2": "generated_snt_sl"}, inplace=True)

In [ ]:
data

,snt_id,simplified_snt,query_id,query_text,doc_id,source_snt,generated_snt_s,generated_snt_sl
0,G11.1_2892036907_1,Current academic and industrial research is in...,G11.1,drones,2892036907,"In the modern era of automation and robotics, ...",Autonomous vehicles are the subject of current...,Self-driving cars are being studied now. This...
1,G11.1_2892036907_2,Drones are increasingly used in the civilian a...,G11.1,drones,2892036907,With the ever increasing number of unmanned ae...,Unmanned aerial vehicles are increasingly used...,Drones are becoming more common in everyday l...
2,G11.1_2892036907_3,Governments set guidelines on the operation ce...,G11.1,drones,2892036907,Due to guidelines set by the governments regar...,Government guidelines exist. Civil drones have...,There are rules set by the government. Drones...
3,G11.1_2892036907_4,Researchers propose data-driven solutions allo...,G11.1,drones,2892036907,In an attempt to achieve the above mentioned t...,UAV autonomy is being achieved. The task invol...,Drones can fly on their own. This job means m...
4,G11.1_2892036907_5,"The algorithm, based on the Inception model, d...",G11.1,drones,2892036907,Derived from the classic image classification ...,The text talks about an image classification a...,The text discusses a method for identifying im...
...,...,...,...,...,...,...,...,...
1083,G01.1_1549686097_2,"Users can access city maps, route planning, na...",G01.1,Digital assistant,1549686097,These services are to be accessed by users wit...,Services are accessible. Users have mobile dev...,Services can be used easily. People use phones...
1084,G01.1_1549686097_3,"Compared to laptops or personal computers, mob...",G01.1,Digital assistant,1549686097,"Mobile end devices, however, do not have compu...",Mobile end devices are mentioned. They lack c...,1. Edward Snowden is a whistleblower who provi...
1085,G01.1_1549686097_4,This shortage can be overcome by using specifi...,G01.1,Digital assistant,1549686097,These deficits can be bypassed by employing sp...,There are deficits. These can be bypassed. Spe...,The study found that children who participate ...
1086,G01.1_1549686097_5,This paper shows the gained experience from de...,G01.1,Digital assistant,1549686097,This paper outlines the experience gained in t...,The paper discusses an experience. It is about...,The article talks about an experiment. It's ab...


In [ ]:
from google.colab import files

data.to_csv('data.csv')
files.download('data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Metrics

In [ ]:
!pip install git+https://github.com/feralvam/easse.git

In [ ]:
import ast
import pandas as pd
import math
from easse.bleu import corpus_bleu
from easse.sari import corpus_sari
from easse.fkgl import corpus_fkgl
import os

In [ ]:
data["ref_snt"] = data.apply(lambda row: [row["simplified_snt"]], axis=1)

In [ ]:
data

,snt_id,simplified_snt,query_id,query_text,doc_id,source_snt,generated_snt_s,generated_snt_sl,SARI,ref_snt
0,G11.1_2892036907_1,Current academic and industrial research is in...,G11.1,drones,2892036907,"In the modern era of automation and robotics, ...",Autonomous vehicles are the subject of current...,Self-driving cars are being studied now. This...,0.0,[Current academic and industrial research is i...
1,G11.1_2892036907_2,Drones are increasingly used in the civilian a...,G11.1,drones,2892036907,With the ever increasing number of unmanned ae...,Unmanned aerial vehicles are increasingly used...,Drones are becoming more common in everyday l...,0.0,[Drones are increasingly used in the civilian ...
2,G11.1_2892036907_3,Governments set guidelines on the operation ce...,G11.1,drones,2892036907,Due to guidelines set by the governments regar...,Government guidelines exist. Civil drones have...,There are rules set by the government. Drones...,0.0,[Governments set guidelines on the operation c...
3,G11.1_2892036907_4,Researchers propose data-driven solutions allo...,G11.1,drones,2892036907,In an attempt to achieve the above mentioned t...,UAV autonomy is being achieved. The task invol...,Drones can fly on their own. This job means m...,0.0,[Researchers propose data-driven solutions all...
4,G11.1_2892036907_5,"The algorithm, based on the Inception model, d...",G11.1,drones,2892036907,Derived from the classic image classification ...,The text talks about an image classification a...,The text discusses a method for identifying im...,0.0,"[The algorithm, based on the Inception model, ..."
...,...,...,...,...,...,...,...,...,...,...
1083,G01.1_1549686097_2,"Users can access city maps, route planning, na...",G01.1,Digital assistant,1549686097,These services are to be accessed by users wit...,Services are accessible. Users have mobile dev...,Services can be used easily. People use phones...,0.0,"[Users can access city maps, route planning, n..."
1084,G01.1_1549686097_3,"Compared to laptops or personal computers, mob...",G01.1,Digital assistant,1549686097,"Mobile end devices, however, do not have compu...",Mobile end devices are mentioned. They lack c...,1. Edward Snowden is a whistleblower who provi...,0.0,"[Compared to laptops or personal computers, mo..."
1085,G01.1_1549686097_4,This shortage can be overcome by using specifi...,G01.1,Digital assistant,1549686097,These deficits can be bypassed by employing sp...,There are deficits. These can be bypassed. Spe...,The study found that children who participate ...,0.0,[This shortage can be overcome by using specif...
1086,G01.1_1549686097_5,This paper shows the gained experience from de...,G01.1,Digital assistant,1549686097,This paper outlines the experience gained in t...,The paper discusses an experience. It is about...,The article talks about an experiment. It's ab...,0.0,[This paper shows the gained experience from d...


### Calcul de BLEU

In [ ]:
data

,snt_id,simplified_snt,query_id,query_text,doc_id,source_snt,generated_snt_s,generated_snt_sl,SARI
0,G11.1_2892036907_1,Current academic and industrial research is in...,G11.1,drones,2892036907,"In the modern era of automation and robotics, ...",Autonomous vehicles are the subject of current...,Self-driving cars are being studied now. This...,0.0
1,G11.1_2892036907_2,Drones are increasingly used in the civilian a...,G11.1,drones,2892036907,With the ever increasing number of unmanned ae...,Unmanned aerial vehicles are increasingly used...,Drones are becoming more common in everyday l...,0.0
2,G11.1_2892036907_3,Governments set guidelines on the operation ce...,G11.1,drones,2892036907,Due to guidelines set by the governments regar...,Government guidelines exist. Civil drones have...,There are rules set by the government. Drones...,0.0
3,G11.1_2892036907_4,Researchers propose data-driven solutions allo...,G11.1,drones,2892036907,In an attempt to achieve the above mentioned t...,UAV autonomy is being achieved. The task invol...,Drones can fly on their own. This job means m...,0.0
4,G11.1_2892036907_5,"The algorithm, based on the Inception model, d...",G11.1,drones,2892036907,Derived from the classic image classification ...,The text talks about an image classification a...,The text discusses a method for identifying im...,0.0
...,...,...,...,...,...,...,...,...,...
1083,G01.1_1549686097_2,"Users can access city maps, route planning, na...",G01.1,Digital assistant,1549686097,These services are to be accessed by users wit...,Services are accessible. Users have mobile dev...,Services can be used easily. People use phones...,0.0
1084,G01.1_1549686097_3,"Compared to laptops or personal computers, mob...",G01.1,Digital assistant,1549686097,"Mobile end devices, however, do not have compu...",Mobile end devices are mentioned. They lack c...,1. Edward Snowden is a whistleblower who provi...,0.0
1085,G01.1_1549686097_4,This shortage can be overcome by using specifi...,G01.1,Digital assistant,1549686097,These deficits can be bypassed by employing sp...,There are deficits. These can be bypassed. Spe...,The study found that children who participate ...,0.0
1086,G01.1_1549686097_5,This paper shows the gained experience from de...,G01.1,Digital assistant,1549686097,This paper outlines the experience gained in t...,The paper discusses an experience. It is about...,The article talks about an experiment. It's ab...,0.0


In [ ]:
BLEU_s = corpus_bleu(sys_sents=data["generated_snt_s"],refs_sents=data["ref_snt"])
BLEU_sl = corpus_bleu(sys_sents=data["generated_snt_sl"],refs_sents=data["ref_snt"])
print(BLEU_s)
print(BLEU_sl)

13.637095534690959
8.777339797465444


### Calcul de SARI

In [ ]:
SARI_s = corpus_sari(
    orig_sents = data["source_snt"],
    sys_sents = data["generated_snt_s"],
    refs_sents = [data["simplified_snt"]])

SARI_sl = corpus_sari(
    orig_sents = data["source_snt"],
    sys_sents = data["generated_snt_sl"],
    refs_sents = [data["simplified_snt"]])

print(SARI_s)
print(SARI_sl)

39.08088294313412
33.73712968050534
